In [1]:
import urllib
from urllib2 import urlopen
import matplotlib.pyplot as plt
from pymongo import MongoClient

%matplotlib inline

db_client = MongoClient()
data = db_client['shopify']['forum']

/Users/brandongarcia/Documents/shadow/shadow_box/lib/python2.7/site-packages/ipykernel/__main__.py:10: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


In [2]:
base_url = 'https://ecommerce.shopify.com/'
forums_url = 'forums'
key = 'forum-navigation-home'

<div style="text-align:center;padding:10px;"><h1>Step 1: Get Forum Categories and Sub-Categories</h1></div>

In [3]:
from bs4 import BeautifulSoup
from collections import defaultdict

forums_home = BeautifulSoup(urlopen(base_url + forums_url))

# Get all discussion forum categories
for cat in forums_home.find_all('ul', {"class":key}):
    
    cat_dict = defaultdict(dict)
    
    # Get each subcategory 
    for item in cat.find_all("li"):

        # Store number of topics and link for each sub_category
        cat_dict[item["class"][-1]]['num_topics'] = int(item.find("div",{"class":"topic-count"}).text.split()[0])
        cat_dict[item["class"][-1]]['url'] = item.find('a').get('href')
        
    data.insert_one({"category":cat["class"][1], "sub_cats":cat_dict})

/Users/brandongarcia/Documents/shadow/shadow_box/lib/python2.7/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 162 of the file /System/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


<div style="text-align:center;padding:10px;"><h1>Step 2: Get "Topic" List for each Sub-Category</h1></div>

In [6]:
for category in data.find():
    try:
        for sub_cat_key in category["sub_cats"]:
            print 'SUBCATEGORY: {sub_cat}'.format(sub_cat=sub_cat_key)
            sub_cat = category["sub_cats"][sub_cat_key]

            # Get number of pages of topics in sub_cat
            topics_page = BeautifulSoup(urlopen(base_url+ sub_cat['url']))
            num_pages = int(topics_page.find('div', {"class":"pagination"}).find_all('a')[-2].text.strip())

            # Links to topics from each page
            category['sub_cats'][sub_cat_key]['topic_urls'] = []
            for page in range(1, num_pages + 1):

                try:
                    topics_page = BeautifulSoup(urlopen(base_url+ sub_cat['url']))
                    category['sub_cats'][sub_cat_key]['topic_urls'] += [topic.find('a').get('href') 
                                                           for topic in topics_page.find_all('span', {"class":"topic"})]
                except Exception as e:
                    print 'PAGE ERROR:', str(e)
                    
                # Status Update
                if page % 5 == 0:
                    print 'PROCESSED {page}/{num_pages} pages'.format(page=page,num_pages=num_pages)
    
        # Update category for db
        data.update_one({"_id":category["_id"]},{'$set':category})
        
    except Exception as e:
        print 'SUB CAT ERROR:', str(e)


SUBCATEGORY: category-accounting-and-taxes
PROCESSED 5/16 pages
PROCESSED 10/16 pages
PROCESSED 15/16 pages
SUBCATEGORY: category-ecommerce-marketing
PROCESSED 5/157 pages
PROCESSED 10/157 pages
PROCESSED 15/157 pages
PROCESSED 20/157 pages
PROCESSED 25/157 pages
PROCESSED 30/157 pages
PROCESSED 35/157 pages
PROCESSED 40/157 pages
PROCESSED 45/157 pages
PROCESSED 50/157 pages
PROCESSED 55/157 pages
PROCESSED 60/157 pages
PROCESSED 65/157 pages
PROCESSED 70/157 pages
PROCESSED 75/157 pages
PROCESSED 80/157 pages
PROCESSED 85/157 pages
PROCESSED 90/157 pages
PROCESSED 95/157 pages
PROCESSED 100/157 pages
PROCESSED 105/157 pages
PROCESSED 110/157 pages
PROCESSED 115/157 pages
PROCESSED 120/157 pages
PROCESSED 125/157 pages
PROCESSED 130/157 pages
PROCESSED 135/157 pages
PROCESSED 140/157 pages
PROCESSED 145/157 pages
PROCESSED 150/157 pages
PROCESSED 155/157 pages
SUBCATEGORY: category-ecommerce-discussion
PROCESSED 5/224 pages
PROCESSED 10/224 pages
PROCESSED 15/224 pages
PROCESSED 20/22

<div style="text-align:center;padding:10px;"><h1>Step 3: Scrape User and Topic Information</h1></div>

In [ ]:


for category in data.find():
    for sub_cat in cateogry["sub-cats"].values():
        print len()